Данный файл служит для создания модели. Модель обучается на искусствено сгенерированных данных, генерирующиеся по тексту из файла 10000-russion-words.txt и шрифтов из папки font. 

In [1]:
from PIL import Image, ImageDraw, ImageFont, ImageOps
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from conf import *

Для правильного подбора параметров в файле conf.py используется сочетание букв "fp". Если они помещаются, то и буквы классических шрифтов поместятся в заданный прямоугольник.

Например:
SIZE = 20                                   # Размер шрифта
W_H = 1.7                                   # Отношение высоты к ширене
DELTA_POSITION = 5                          # Максимальный отступ сверху


In [2]:
size_w = 20
size_h = 20
size = 15
img = Image.new('L', (size_w, size_h), color = 255)
 
fnt = ImageFont.truetype('font/TimesNewRomanCyr_Regular.ttf', size)
d = ImageDraw.Draw(img)
d.text((0, -3), "fp", font=fnt, fill=0)
img

Следующие две функции необходимы для генерации данных. 
- get_img_work для генерации одной картинки с заданным текстом;
- get_dataset сам массив обучающих и тестовых данных.

При генерации берутся разные шрифты, разные типы шрифтов (что и отличает программа),различные смещения (DELTA_POSITION число смещений по OX, столько же по OY), различные слова из файла.

In [3]:
def get_img_word(text_word, font, dx, dy):
    '''
    Создаем картинку слова в виде массива пикселей row размера M,
    а также картинку img_rez как Image объект.
    '''
    
    # Находим размеры слова по числу букв
    n=len(text_word)
    size_w = round(SIZE/W_H*n + DELTA_POSITION)
    size_h = H_SIZE
    
    # Рисование текста
    img = Image.new('L', (size_w, size_h), color = 0)
    fnt = ImageFont.truetype(font, SIZE) 
    d = ImageDraw.Draw(img)
    d.text((START_X+dx, START_Y+dy), text_word, font=fnt, fill=255)

    # Подгоняем размер под средний
    crop_image = img.crop((size_w-MEAN_W_SIZE_WORD, 0, size_w, size_h))
    img_rez = ImageOps.invert(crop_image)
    row = np.reshape(img_rez, M)/255
    return (row, img_rez)

In [4]:
def get_dataset():
    """
    Генерируем данные для обучения модели. Слова из файла с разными шрифтами,
    прописаными выше. На выходе получаем 2 набора: 
    (X_train, y_train) для обучения, (X_test, y_test) для проверки 
    """
    # https://github.com/hingston/russian/blob/master/10000-russian-words.txt
    words = []
    with open("10000-russian-words.txt", "r", encoding="utf-8") as f:
        for line in f:
            words.append(line[:-1]) #Без символа '\n'

    words=words[:COUNT_WORDS]
    positions_N = DELTA_POSITION**2

    count_words = len(words)
    count_name_fonts = len(NAME_FONTS)
    count_type_fonts = len(TYPE_FONTS)

    num_type_fonts = range(count_type_fonts)

    n = count_words*count_name_fonts*count_type_fonts*positions_N

    X = np.zeros((n, M))
    y = np.zeros((n, 1))
    k=0

    for name_font in NAME_FONTS:
        for index_type_font in range(count_type_fonts):
            fnt = f'font/{name_font}_{TYPE_FONTS[index_type_font]}.ttf'
            for position in range(positions_N):
                for i in range(count_words):
                    dx = position//DELTA_POSITION
                    dy = position%DELTA_POSITION
                    row, _ = get_img_word(words[i],fnt, dx, dy)
                    X[k*count_words+i, :] = row
                    y[k*count_words+i] = index_type_font
                k += 1
    X = pd.DataFrame(X)
    y = pd.DataFrame(y)
    shuffle_index = np.random.permutation(n)

    X.reindex(index=shuffle_index)
    y.reindex(index=shuffle_index)
    
    index_start_test = int(n/4*3)
    X_train, X_test = X[:index_start_test], X[index_start_test:]
    y_train, y_test = y[:index_start_test], y[index_start_test:]

    return (X_train, y_train, X_test, y_test)

Непосредственно настройка модели, параметры взяты из примера (https://www.tensorflow.org/tutorials/keras/classification)

In [5]:
# Получение данных
X_train, y_train, X_test, y_test = get_dataset()

In [6]:
# СОЗДАНИЕ МОДЕЛИ
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(M,)), # Запись данных в строку
    tf.keras.layers.Dense(128, activation='relu'), # Слой с 128 нейронами
    tf.keras.layers.Dense(len(TYPE_FONTS)) # Массив логитов длиной 2-3
])

In [7]:
# НАСТРОЙКА МОДЕЛИ
model.compile(optimizer='adam', # Про методы https://habr.com/ru/post/318970/
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']) # Метрика, в данном случае точность

In [8]:
# Обучение и проверка результата
model.fit(X_train.to_numpy(), y_train.to_numpy(), epochs=10)
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=len(TYPE_FONTS))

print('\nTest accuracy:', test_acc)

Epoch 1/10
2110/2110 [==============================] - 5s 2ms/step - loss: 0.8168 - accuracy: 0.6020
Epoch 2/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.6238 - accuracy: 0.7020
Epoch 3/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.5555 - accuracy: 0.7330
Epoch 4/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.5198 - accuracy: 0.7552
Epoch 5/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4946 - accuracy: 0.7660
Epoch 6/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4770 - accuracy: 0.7767
Epoch 7/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4612 - accuracy: 0.7859
Epoch 8/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4478 - accuracy: 0.7943
Epoch 9/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4383 - accuracy: 0.8010
Epoch 10/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.429

Модель сохраняется в файле указаном в NAME_MODEL. Этот файл используется в font_style_classifier

In [9]:
# Сохранение модели
model.save(NAME_MODEL)

INFO:tensorflow:Assets written to: model_3\assets
